In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Визначення середовища
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
except:
    IN_COLAB = False
    print("💻 Running locally")

# Завантаження датасету
if IN_COLAB:
    # В Colab - завантажуємо з GitHub
    dataset_path = 'https://raw.githubusercontent.com/Sarmatae685/Intelligent-Data-Analysis/main/lab1-data-preprocessing/CloudWatch_Traffic_Web_Attack_.csv'
else:
    # Локально - файл поруч
    dataset_path = 'CloudWatch_Traffic_Web_Attack_.csv'

df = pd.read_csv(dataset_path)
print(f"✅ Dataset loaded: {len(df)} records")
df.head(5)

ModuleNotFoundError: No module named 'google'

In [ ]:
# Перевірка на наявність пропущених значень
missing_values = df.isna().sum()
print("Пропущені значення по стовпцях:\n", missing_values)

In [ ]:
import numpy as np
# Заповнення числових пропущених значень середнім
numeric_columns = df.select_dtypes(include=np.number).columns

df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

# Заповнення категоріальних пропущених значень найбільш поширеними
categorical_columns = df.select_dtypes(include='object').columns

for col in categorical_columns:
    most_frequent = df[col].mode()[0]
    df[col].fillna(most_frequent, inplace=True)

In [ ]:
# Перевірка після заповнення
missing_values = df.isna().sum()
print("Пропущені значення по стовпцях після заповнення:\n", missing_values)

In [ ]:
# Виявлення та видалення викидів за Z-score
from scipy.stats import zscore
import numpy as np

# Розрахунок Z-score для стовпця 'bytes_in'
df['zscore-bytes_in'] = zscore(df['bytes_in'])
# Поріг для виявлення викидів
threshold = 2
# Позначення викидів у новому стовпці
df['outliers'] = np.where(
    (df['zscore-bytes_in'] > threshold) | (df['zscore-bytes_in'] < -threshold),
    True,
    False
)

In [ ]:
# Видалення викидів
df_cleaned = df.drop(df[df['outliers'] == True].index)
print("Розмір після видалення викидів:", df_cleaned.shape)

In [ ]:
# Нормалізація і масштабування

# Застосування нормалізації Min-Max без наукових позначень у вихідних даних
bytes_in_min_max = (df['bytes_in'] - df['bytes_in'].min()) / (df['bytes_in'].max() - df['bytes_in'].min())
print("Min-Max Normalization:\n", bytes_in_min_max.head(12).apply(lambda x: f"{x:.6f}"))

# Логарифмічне масштабування
bytes_in_log = df['bytes_in'].apply(lambda x: np.log(x) if x != 0 else 0)
print("\nЛогарифмічне масштабування:\n", bytes_in_log.head(12).apply(lambda x: f"{x:.6f}"))

# Обрізання значень (Clipping) з використанням vmax і vmin
vmax = 17748
vmin = 0
df['bytes_in_clipped'] = df['bytes_in'].apply(lambda x: vmax if x > vmax else vmin if x < vmin else x)

# Відображення результатів обрізання
print("Обрізання значень:\n", df[['bytes_in', 'bytes_in_clipped']].head(12).applymap(lambda x: f"{x:.0f}"))

In [ ]:
# Масштабування однієї ознаки (SFS) в стовпці 'bytes_in' до діапазону від 0 до 1
if 'bytes_in' in df.columns:
    df['bytes_in_sfs'] = df['bytes_in'] / df['bytes_in'].max()
else:
    df['Error'] = "Column 'bytes_in' not found in the dataset."

# Відображення масштабованих значень
df[['bytes_in', 'bytes_in_sfs']].head(10).applymap(lambda x: f"{x:.6f}" if pd.notnull(x) else x)